# description
Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

In their first Kaggle competition, Rossmann is challenging you to predict 6 weeks of daily sales for 1,115 stores located across Germany. Reliable sales forecasts enable store managers to create effective staff schedules that increase productivity and motivation. By helping Rossmann create a robust prediction model, you will help store managers stay focused on what’s most important to them: their customers and their teams!

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout, Input, Concatenate,LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Model
import os
import tarfile
import numpy as np

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd
import keras

In [3]:
import re

# 载入文件

Data fields
Most of the fields are self-explanatory. The following are descriptions for those that aren't.
Id - an Id that represents a (Store, Date) duple within the test set
Store - a unique Id for each store
Sales - the turnover for any given day (this is what you are predicting)
Customers - the number of customers on a given day
Open - an indicator for whether the store was open: 0 = closed, 1 = open
StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
StoreType - differentiates between 4 different store models: a, b, c, d
Assortment - describes an assortment level: a = basic, b = extra, c = extended
CompetitionDistance - distance in meters to the nearest competitor store
CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
Promo - indicates whether a store is running a promo on that day
Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

In [4]:
path = r'./rossmann/'

In [5]:
table_names = ['train', 'store', 'store_states', 'state_names',  'googletrend', 'weather', 'test']  #需要用到的表名

In [6]:
tables = [pd.read_csv(path+fname+r'.csv', low_memory=False) for fname in table_names]  #将需要用到的表存入tables

In [7]:
from IPython.display import HTML, display

In [8]:
for t in tables:             
    display(t.head())

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


,Store,State
0,1,HE
1,2,TH
2,3,NW
3,4,BE
4,5,SN


,StateName,State
0,BadenWuerttemberg,BW
1,Bayern,BY
2,Berlin,BE
3,Brandenburg,BB
4,Bremen,HB


,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67


,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


# 特征工程

In [9]:
train, store, store_states, state_names, googletrend, weather, test = tables   #将tables中的表分别赋值给不同的变量

In [10]:
len(train),len(test)   #训练集和测试集的大小

(1017209, 41088)

我们分析各个表中的特征，找出连续特征和离散特征
train:
    离散特征：Store, DayOfWeek,Open, Promo,StateHoliday,SchoolHoliday
    连续特征：Sales, Customers,Date
store:
    离散特征：Store, StoreType,Assortment,Promo2,PromoInterval
    连续特征：Sales, Customers,Date,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,
           Promo2SinceWeek,Promo2SinceWeek,Promo2SinceYear
store_states:
    离散特征：Store,State
    连续特征：无
state_names:
    离散特征：StateName,State
    连续特征：无
googletrend:
    离散特征：file
    连续特征：week, trend
weather：
    离散特征：file,Events 
    连续特征：Date, Max_TemperatureC, Mean_TemperatureC, Min_TemperatureC, Dew_PointC,MeanDew_PointC, 
          Min_DewpointC, Max_Humidity,Mean_Humidity, Min_Humidity, Max_Sea_Level_PressurehPa,Mean_Sea_Level_PressurehPa, 
          Min_Sea_Level_PressurehPa, Max_VisibilityKm, Mean_VisibilityKm,Min_VisibilitykM, Max_Wind_SpeedKm_h, Mean_Wind_SpeedKm_h,
          ,Max_Wind_SpeedKm_h, Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,WindDirDegrees 
test:
    离散特征：Id,Store,DayOfWeek, Open, Promo,StateHoliday,SchoolHoliday
    连续特征：Date
      
          

    
    

In [11]:
#将StateHoliday转成boolean形
train.StateHoliday = train.StateHoliday!='0'
test.StateHoliday = test.StateHoliday!='0'

In [12]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,False,1
1,2,5,2015-07-31,6064,625,1,1,False,1
2,3,5,2015-07-31,8314,821,1,1,False,1
3,4,5,2015-07-31,13995,1498,1,1,False,1
4,5,5,2015-07-31,4822,559,1,1,False,1


## 

In [13]:
# merge的参数

# on：列名，join用来对齐的那一列的名字，用到这个参数的时候一定要保证左表和右表用来对齐的那一列都有相同的列名。如果未指定，则选择两个表的交集

# left_on：左表对齐的列，可以是列名，也可以是和dataframe同样长度的arrays。

# right_on：右表对齐的列，可以是列名，也可以是和dataframe同样长度的arrays。

# left_index/ right_index: 如果是True的haunted以index作为对齐的key

# how：数据融合的方法，inner: 显示两表的交集，outer：显示两表的并集，left: 只显示左表的内容，right: 只显示右表的内容。

# sort：根据dataframe合并的keys按字典顺序排序，默认是，如果置false可以提高表现。

def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: 
        right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))


In [14]:
#左表是weather,右表是state_names
weather = join_df(weather, state_names, "file", "StateName")


weather = weather + state_names

In [15]:
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2]
googletrend.loc[googletrend.State=='NI', "State"] = 'HB,NI'  #loc(row,column)选取特定的行列，赋值

In [16]:
googletrend.head()

,file,week,trend,Date,State
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96,2012-12-02,SN
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95,2012-12-09,SN
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91,2012-12-16,SN
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48,2012-12-23,SN
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67,2012-12-30,SN


In [17]:
def add_datepart(df, fldname, drop=True, time=False):   #分离日期数据
    """add_datepart converts a column of df from a datetime64 to many columns containing
    the information from the date. This applies changes inplace.
    Parameters:
    -----------
    df: A pandas data frame. df gain several new columns.
    fldname: A string that is the name of the date column you wish to expand.
        If it is not a datetime64 series, it will be converted to one with pd.to_datetime.
    drop: If true then the original date column will be removed.
    time: If true time features: Hour, Minute, Second will be added.
    Examples:
    ---------
    >>> df = pd.DataFrame({ 'A' : pd.to_datetime(['3/11/2000', '3/12/2000', '3/13/2000'], infer_datetime_format=False) })
    >>> df
        A
    0   2000-03-11
    1   2000-03-12
    2   2000-03-13
    >>> add_datepart(df, 'A')
    >>> df
        AYear AMonth AWeek ADay ADayofweek ADayofyear AIs_month_end AIs_month_start AIs_quarter_end AIs_quarter_start AIs_year_end AIs_year_start AElapsed
    0   2000  3      10    11   5          71         False         False           False           False             False        False          952732800
    1   2000  3      10    12   6          72         False         False           False           False             False        False          952819200
    2   2000  3      11    13   0          73         False         False           False           False             False        False          952905600
    """
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: 
        attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: 
        df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: 
        df.drop(fldname, axis=1, inplace=True)

In [18]:
add_datepart(weather, "Date", drop=False)
add_datepart(googletrend, "Date", drop=False)
add_datepart(train, "Date", drop=False)
add_datepart(test, "Date", drop=False)


In [19]:
weather

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,1,1,1,False,True,False,True,False,True,1356998400
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,2,2,2,False,False,False,False,False,False,1357084800
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,3,3,3,False,False,False,False,False,False,1357171200
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,4,4,4,False,False,False,False,False,False,1357257600
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,5,5,5,False,False,False,False,False,False,1357344000
5,NordrheinWestfalen,2013-01-06,9,8,7,8,7,6,100,92,...,6,6,6,False,False,False,False,False,False,1357430400
6,NordrheinWestfalen,2013-01-07,9,8,7,8,7,5,98,88,...,7,0,7,False,False,False,False,False,False,1357516800
7,NordrheinWestfalen,2013-01-08,8,7,5,6,5,4,100,87,...,8,1,8,False,False,False,False,False,False,1357603200
8,NordrheinWestfalen,2013-01-09,6,6,5,6,4,3,100,91,...,9,2,9,False,False,False,False,False,False,1357689600
9,NordrheinWestfalen,2013-01-10,6,4,3,4,3,1,96,86,...,10,3,10,False,False,False,False,False,False,1357776000


In [20]:
trend_de = googletrend[googletrend.file == 'Rossmann_DE'] #提取出没有地区信息的数据

In [21]:
trend_de

,file,week,trend,Date,State,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
148,Rossmann_DE,2012-12-02 - 2012-12-08,77,2012-12-02,None,2012,12,48,2,6,337,False,False,False,False,False,False,1354406400
149,Rossmann_DE,2012-12-09 - 2012-12-15,76,2012-12-09,None,2012,12,49,9,6,344,False,False,False,False,False,False,1355011200
150,Rossmann_DE,2012-12-16 - 2012-12-22,85,2012-12-16,None,2012,12,50,16,6,351,False,False,False,False,False,False,1355616000
151,Rossmann_DE,2012-12-23 - 2012-12-29,59,2012-12-23,None,2012,12,51,23,6,358,False,False,False,False,False,False,1356220800
152,Rossmann_DE,2012-12-30 - 2013-01-05,61,2012-12-30,None,2012,12,52,30,6,365,False,False,False,False,False,False,1356825600
153,Rossmann_DE,2013-01-06 - 2013-01-12,62,2013-01-06,None,2013,1,1,6,6,6,False,False,False,False,False,False,1357430400
154,Rossmann_DE,2013-01-13 - 2013-01-19,51,2013-01-13,None,2013,1,2,13,6,13,False,False,False,False,False,False,1358035200
155,Rossmann_DE,2013-01-20 - 2013-01-26,56,2013-01-20,None,2013,1,3,20,6,20,False,False,False,False,False,False,1358640000
156,Rossmann_DE,2013-01-27 - 2013-02-02,50,2013-01-27,None,2013,1,4,27,6,27,False,False,False,False,False,False,1359244800
157,Rossmann_DE,2013-02-03 - 2013-02-09,58,2013-02-03,None,2013,2,5,3,6,34,False,False,False,False,False,False,1359849600


In [22]:
store = join_df(store, store_states, "Store")

store = store + store_states

In [23]:
joined = join_df(train, store, "Store")
joined_test = join_df(test, store, "Store")

joined = train+store

In [24]:
joined.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State
0,1,5,2015-07-31,5263,555,1,1,False,1,2015,...,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,HE
1,2,5,2015-07-31,6064,625,1,1,False,1,2015,...,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",TH
2,3,5,2015-07-31,8314,821,1,1,False,1,2015,...,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",NW
3,4,5,2015-07-31,13995,1498,1,1,False,1,2015,...,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN,BE
4,5,5,2015-07-31,4822,559,1,1,False,1,2015,...,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN,SN


In [25]:
joined = join_df(joined, googletrend, ["State","Year", "Week"])
joined_test = join_df(joined_test, googletrend, ["State","Year", "Week"])


joined = joined + googletrend

In [26]:
joined.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,Day_y,Dayofweek_y,Dayofyear_y,Is_month_end_y,Is_month_start_y,Is_quarter_end_y,Is_quarter_start_y,Is_year_end_y,Is_year_start_y,Elapsed_y
0,1,5,2015-07-31,5263,555,1,1,False,1,2015,...,2,6,214,False,False,False,False,False,False,1438473600
1,2,5,2015-07-31,6064,625,1,1,False,1,2015,...,2,6,214,False,False,False,False,False,False,1438473600
2,3,5,2015-07-31,8314,821,1,1,False,1,2015,...,2,6,214,False,False,False,False,False,False,1438473600
3,4,5,2015-07-31,13995,1498,1,1,False,1,2015,...,2,6,214,False,False,False,False,False,False,1438473600
4,5,5,2015-07-31,4822,559,1,1,False,1,2015,...,2,6,214,False,False,False,False,False,False,1438473600


In [27]:
joined = joined.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
joined_test = joined_test.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))

In [28]:
joined

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,Day_DE,Dayofweek_DE,Dayofyear_DE,Is_month_end_DE,Is_month_start_DE,Is_quarter_end_DE,Is_quarter_start_DE,Is_year_end_DE,Is_year_start_DE,Elapsed_DE
0,1,5,2015-07-31,5263,555,1,1,False,1,2015,...,2,6,214,False,False,False,False,False,False,1438473600
1,2,5,2015-07-31,6064,625,1,1,False,1,2015,...,2,6,214,False,False,False,False,False,False,1438473600
2,3,5,2015-07-31,8314,821,1,1,False,1,2015,...,2,6,214,False,False,False,False,False,False,1438473600
3,4,5,2015-07-31,13995,1498,1,1,False,1,2015,...,2,6,214,False,False,False,False,False,False,1438473600
4,5,5,2015-07-31,4822,559,1,1,False,1,2015,...,2,6,214,False,False,False,False,False,False,1438473600
5,6,5,2015-07-31,5651,589,1,1,False,1,2015,...,2,6,214,False,False,False,False,False,False,1438473600
6,7,5,2015-07-31,15344,1414,1,1,False,1,2015,...,2,6,214,False,False,False,False,False,False,1438473600
7,8,5,2015-07-31,8492,833,1,1,False,1,2015,...,2,6,214,False,False,False,False,False,False,1438473600
8,9,5,2015-07-31,8565,687,1,1,False,1,2015,...,2,6,214,False,False,False,False,False,False,1438473600
9,10,5,2015-07-31,7185,681,1,1,False,1,2015,...,2,6,214,False,False,False,False,False,False,1438473600


In [29]:
joined = join_df(joined, weather, ["State","Date"])
joined_test = join_df(joined_test, weather, ["State","Date"])

joined =  joined +weather

In [30]:
joined.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,Day_y,Dayofweek_y,Dayofyear_y,Is_month_end_y,Is_month_start_y,Is_quarter_end_y,Is_quarter_start_y,Is_year_end_y,Is_year_start_y,Elapsed_y
0,1,5,2015-07-31,5263,555,1,1,False,1,2015,...,31,4,212,True,False,False,False,False,False,1438300800
1,2,5,2015-07-31,6064,625,1,1,False,1,2015,...,31,4,212,True,False,False,False,False,False,1438300800
2,3,5,2015-07-31,8314,821,1,1,False,1,2015,...,31,4,212,True,False,False,False,False,False,1438300800
3,4,5,2015-07-31,13995,1498,1,1,False,1,2015,...,31,4,212,True,False,False,False,False,False,1438300800
4,5,5,2015-07-31,4822,559,1,1,False,1,2015,...,31,4,212,True,False,False,False,False,False,1438300800


In [31]:
#删掉重复行
for df in (joined, joined_test):
    for c in df.columns:
        if c.endswith('_y'):
            if c in df.columns: 
                df.drop(c, inplace=True, axis=1)

In [32]:
joined.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,StateName
0,1,5,2015-07-31,5263,555,1,1,False,1,2015,...,15.0,10.0,24,11,NaN,0.0,1.0,Fog,13,Hessen
1,2,5,2015-07-31,6064,625,1,1,False,1,2015,...,10.0,10.0,14,11,NaN,0.0,4.0,Fog,309,Thueringen
2,3,5,2015-07-31,8314,821,1,1,False,1,2015,...,14.0,10.0,14,5,NaN,0.0,2.0,Fog,354,NordrheinWestfalen
3,4,5,2015-07-31,13995,1498,1,1,False,1,2015,...,10.0,10.0,23,16,NaN,0.0,6.0,NaN,282,Berlin
4,5,5,2015-07-31,4822,559,1,1,False,1,2015,...,10.0,10.0,14,11,NaN,0.0,4.0,NaN,290,Sachsen


## 缺失值填充

In [33]:
#用设定的值填充缺失值，年填为1900，周填为1
for df in (joined,joined_test):
    df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)

In [34]:
df['CompetitionOpenSinceYear'][:10]

0    2008
1    2006
2    2013
3    2014
4    2000
5    2009
6    2011
7    1900
8    1900
9    2014
Name: CompetitionOpenSinceYear, dtype: int32

In [35]:
df['CompetitionOpenSinceMonth'][:10]

0     9
1    12
2     4
3    10
4     8
5     9
6    11
7     1
8     1
9     3
Name: CompetitionOpenSinceMonth, dtype: int32

In [36]:
df['Promo2SinceYear'][:10]

0    1900
1    2011
2    1900
3    1900
4    1900
5    1900
6    2012
7    2010
8    2009
9    2011
Name: Promo2SinceYear, dtype: int32

In [37]:
 df['Promo2SinceWeek'][:10]

0     1
1    14
2     1
3     1
4     1
5     1
6     1
7    13
8    45
9    40
Name: Promo2SinceWeek, dtype: int32

In [38]:
#将CompetitionOpenSinceYear，CompetitionOpenSinceMonth组合成具体的日期
for df in (joined,joined_test):
    df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                     month=df.CompetitionOpenSinceMonth, day=15))
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days

In [39]:
df["CompetitionOpenSince"][:10]

0   2008-09-15
1   2006-12-15
2   2013-04-15
3   2014-10-15
4   2000-08-15
5   2009-09-15
6   2011-11-15
7   1900-01-15
8   1900-01-15
9   2014-03-15
Name: CompetitionOpenSince, dtype: datetime64[ns]

In [40]:
for df in (joined,joined_test):
    df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear<1990, "CompetitionOpenSinceYear"] = 0

In [41]:
#将CompetitionDaysOpen转化为CompetitionMonthsOpen，一月按30天算，最大是24个月，超过24个月的按照24个月计算
for df in (joined,joined_test):
    df["CompetitionMonthsOpen"] = df["CompetitionDaysOpen"]//30
    df.loc[df.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24
joined.CompetitionMonthsOpen.unique()

array([24,  3, 19,  9, 16, 17,  7, 15, 22, 11, 13,  2, 23,  0, 12,  4, 10,
        1, 14, 20,  8, 18,  6, 21,  5], dtype=int64)

In [42]:
from isoweek import Week

In [43]:
for df in (joined,joined_test):
    df["Promo2Since"] = pd.to_datetime(df.apply(lambda x: Week(
        x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1).astype(pd.datetime))
    df["Promo2Days"] = df.Date.subtract(df["Promo2Since"]).dt.days

In [44]:
df["Promo2Days"][:10]

0    42262
1     1627
2    42262
3    42262
4    42262
5    42262
6     1354
7     1998
8     2145
9     1445
Name: Promo2Days, dtype: int64

In [45]:
df.Promo2SinceYear

0        1900
1        2011
2        1900
3        1900
4        1900
5        1900
6        2012
7        2010
8        2009
9        2011
10       2011
11       1900
12       2011
13       2014
14       2009
15       2012
16       1900
17       2011
18       1900
19       2011
20       1900
21       2014
22       1900
23       2009
24       1900
25       2012
26       2014
27       1900
28       2013
29       2009
         ... 
41058    2010
41059    2011
41060    2009
41061    2014
41062    2013
41063    2010
41064    2011
41065    1900
41066    2013
41067    1900
41068    1900
41069    2014
41070    2014
41071    2014
41072    1900
41073    2013
41074    2011
41075    1900
41076    2014
41077    2013
41078    2011
41079    2013
41080    2013
41081    2010
41082    2012
41083    2013
41084    1900
41085    1900
41086    1900
41087    2012
Name: Promo2SinceYear, Length: 41088, dtype: int32

In [46]:
# for df in (joined,joined_test):
#     df.loc[df.Promo2Days<0, "Promo2Days"] = 0
#     df.loc[df.Promo2SinceYear<1990, "Promo2Days"] = 0
#     df["Promo2Weeks"] = df["Promo2Days"]//7
#     df.loc[df.Promo2Weeks<0, "Promo2Weeks"] = 0
#     df.loc[df.Promo2Weeks>25, "Promo2Weeks"] = 25
#     df.Promo2Weeks.unique()

for df in (joined,joined_test):
    df.loc[df.Promo2Days<0, "Promo2Days"] = 0
    df.loc[df.Promo2SinceYear<1990, "Promo2Days"] = 0
    df["Promo2Weeks"] = df["Promo2Days"]//7
    df.loc[df.Promo2Weeks<0, "Promo2Weeks"] = 0
    df.loc[df.Promo2Weeks>25, "Promo2Weeks"] = 25
    df.Promo2Weeks.unique()

In [47]:
df.Promo2SinceYear[:10]

0    1900
1    2011
2    1900
3    1900
4    1900
5    1900
6    2012
7    2010
8    2009
9    2011
Name: Promo2SinceYear, dtype: int32

In [48]:
joined.to_feather( r'./rossmann/'+'joined')  #读出文件
joined_test.to_feather(r'./rossmann/'+'joined_test')

In [49]:
joined.to_csv(r'./rossmann/joined.csv', encoding = 'utf-8', index = False)

## 提取时间序列特征，可以通过行与行之间的关系来提取时间序列的特征，比如
1. 计算一列的平均值
2. 距离下一个观测值的时间间隔
3. 距离上一观测值的时间间隔

In [50]:
def get_elapsed(fld, pre):
    day1 = np.timedelta64(1, 'D')  #timedelta时间格式，单位为1天
    last_date = np.datetime64()    #datetime64时间格式：tetime64('2018-09-01T00:00:00.000000')
    last_store = 0
    res = []

    for s,v,d in zip(df.Store.values,df[fld].values, df.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: 
            last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    df[pre+fld] = res

In [51]:
columns = ["Date", "Store", "Promo", "StateHoliday", "SchoolHoliday"]

In [52]:
df = train[columns].append(test[columns])

In [53]:
fld = 'SchoolHoliday'

df = df.sort_values(['Store', 'Date'])



In [54]:
df.head()

,Date,Store,Promo,StateHoliday,SchoolHoliday
1016095,2013-01-01,1,0,True,1
1014980,2013-01-02,1,0,False,1
1013865,2013-01-03,1,0,False,1
1012750,2013-01-04,1,0,False,1
1011635,2013-01-05,1,0,False,1


In [55]:
#After表示距离上一个节假日的时间
#Before表示距离下一个节假日的时间
get_elapsed(fld, 'After')

df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [56]:
df.head()

,Date,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday
0,2015-09-17,1,1,False,0,13.0,NaN
856,2015-09-16,1,1,False,0,12.0,NaN
1712,2015-09-15,1,1,False,0,11.0,NaN
2568,2015-09-14,1,1,False,0,10.0,NaN
3424,2015-09-13,1,0,False,0,9.0,NaN


In [57]:
fld = 'StateHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [58]:
fld = 'Promo'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [59]:
#将时间特征Date变为行特征Row

df = df.set_index("Date")  

In [60]:
df.head()

,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo
Date,,,,,,,,,,
2015-09-17,1,1,False,0,13.0,NaN,105.0,NaN,0.0,0.0
2015-09-16,1,1,False,0,12.0,NaN,104.0,NaN,0.0,0.0
2015-09-15,1,1,False,0,11.0,NaN,103.0,NaN,0.0,0.0
2015-09-14,1,1,False,0,10.0,NaN,102.0,NaN,0.0,0.0
2015-09-13,1,0,False,0,9.0,NaN,101.0,NaN,9.0,-1.0


In [61]:
columns = ['SchoolHoliday', 'StateHoliday', 'Promo']

In [62]:
for o in ['Before', 'After']:
    for p in columns:
        a = o+p
        df[a] = df[a].fillna(0).astype(int)

In [63]:
df

,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo
Date,,,,,,,,,,
2015-09-17,1,1,False,0,13,0,105,0,0,0
2015-09-16,1,1,False,0,12,0,104,0,0,0
2015-09-15,1,1,False,0,11,0,103,0,0,0
2015-09-14,1,1,False,0,10,0,102,0,0,0
2015-09-13,1,0,False,0,9,0,101,0,9,-1
2015-09-12,1,0,False,0,8,0,100,0,8,-2
2015-09-11,1,0,False,0,7,0,99,0,7,-3
2015-09-10,1,0,False,0,6,0,98,0,6,-4
2015-09-09,1,0,False,0,5,0,97,0,5,-5


In [64]:
bwd = df[['Store']+columns].sort_index().groupby("Store").rolling(7, min_periods=1).sum()

In [65]:
bwd

Store  SchoolHoliday  StateHoliday  Promo
Store Date                                                  
1     2013-01-01     1.0            1.0           1.0    0.0
      2013-01-02     2.0            2.0           1.0    0.0
      2013-01-03     3.0            3.0           1.0    0.0
      2013-01-04     4.0            4.0           1.0    0.0
      2013-01-05     5.0            5.0           1.0    0.0
      2013-01-06     6.0            6.0           1.0    0.0
      2013-01-07     7.0            7.0           1.0    1.0
      2013-01-08     7.0            7.0           0.0    2.0
      2013-01-09     7.0            7.0           0.0    3.0
      2013-01-10     7.0            7.0           0.0    4.0
      2013-01-11     7.0            7.0           0.0    5.0
      2013-01-12     7.0            6.0           0.0    5.0
      2013-01-13     7.0            5.0           0.0    5.0
      2013-01-14     7.0            4.0           0.0    4.0
      2013-01-15     7.0            3.0           0.0    3.0
      2013-01-16     7.0            2.0           0.0    2.0
      2013-01-17     7.0            1.0           0.0    1.0
      2013-01-18     7.0            0.0           0.0    0.0
      2013-01-19     7.0            0.0           0.0    0.0
      2013-01-20     7.0            0.0           0.0    0.0
      2013-01-21     7.0            0.0           0.0    1.0
      2013-01-22     7.0            0.0           0.0    2.0
      2013-01-23     7.0            0.0           0.0    3.0
      2013-01-24     7.0            0.0           0.0    4.0
      2013-01-25     7.0            0.0           0.0    5.0
      2013-01-26     7.0            0.0           0.0    5.0
      2013-01-27     7.0            0.0           0.0    5.0
      2013-01-28     7.0            0.0           0.0    4.0
      2013-01-29     7.0            0.0           0.0    3.0
      2013-01-30     7.0            0.0           0.0    2.0
...                  ...            ...           ...    ...
1115  2015-08-19  7805.0            7.0           0.0    3.0
      2015-08-20  7805.0            7.0           0.0    4.0
      2015-08-21  7805.0            7.0           0.0    5.0
      2015-08-22  7805.0            7.0           0.0    5.0
      2015-08-23  7805.0            7.0           0.0    5.0
      2015-08-24  7805.0            7.0           0.0    4.0
      2015-08-25  7805.0            7.0           0.0    3.0
      2015-08-26  7805.0            7.0           0.0    2.0
      2015-08-27  7805.0            7.0           0.0    1.0
      2015-08-28  7805.0            7.0           0.0    0.0
      2015-08-29  7805.0            7.0           0.0    0.0
      2015-08-30  7805.0            7.0           0.0    0.0
      2015-08-31  7805.0            7.0           0.0    1.0
      2015-09-01  7805.0            7.0           0.0    2.0
      2015-09-02  7805.0            7.0           0.0    3.0
      2015-09-03  7805.0            7.0           0.0    4.0
      2015-09-04  7805.0            7.0           0.0    5.0
      2015-09-05  7805.0            6.0           0.0    5.0
      2015-09-06  7805.0            5.0           0.0    5.0
      2015-09-07  7805.0            4.0           0.0    4.0
      2015-09-08  7805.0            3.0           0.0    3.0
      2015-09-09  7805.0            2.0           0.0    2.0
      2015-09-10  7805.0            1.0           0.0    1.0
      2015-09-11  7805.0            0.0           0.0    0.0
      2015-09-12  7805.0            0.0           0.0    0.0
      2015-09-13  7805.0            0.0           0.0    0.0
      2015-09-14  7805.0            0.0           0.0    1.0
      2015-09-15  7805.0            0.0           0.0    2.0
      2015-09-16  7805.0            0.0           0.0    3.0
      2015-09-17  7805.0            0.0           0.0    4.0

[1058297 rows x 4 columns]

In [66]:
fwd = df[['Store']+columns].sort_index(ascending=False
                                      ).groupby("Store").rolling(7, min_periods=1).sum()

In [67]:
fwd

Store  SchoolHoliday  StateHoliday  Promo
Store Date                                                  
1     2015-09-17     1.0            0.0           0.0    1.0
      2015-09-16     2.0            0.0           0.0    2.0
      2015-09-15     3.0            0.0           0.0    3.0
      2015-09-14     4.0            0.0           0.0    4.0
      2015-09-13     5.0            0.0           0.0    4.0
      2015-09-12     6.0            0.0           0.0    4.0
      2015-09-11     7.0            0.0           0.0    4.0
      2015-09-10     7.0            0.0           0.0    3.0
      2015-09-09     7.0            0.0           0.0    2.0
      2015-09-08     7.0            0.0           0.0    1.0
      2015-09-07     7.0            0.0           0.0    0.0
      2015-09-06     7.0            0.0           0.0    0.0
      2015-09-05     7.0            0.0           0.0    0.0
      2015-09-04     7.0            1.0           0.0    1.0
      2015-09-03     7.0            2.0           0.0    2.0
      2015-09-02     7.0            3.0           0.0    3.0
      2015-09-01     7.0            4.0           0.0    4.0
      2015-08-31     7.0            5.0           0.0    5.0
      2015-08-30     7.0            6.0           0.0    5.0
      2015-08-29     7.0            7.0           0.0    5.0
      2015-08-28     7.0            7.0           0.0    4.0
      2015-08-27     7.0            7.0           0.0    3.0
      2015-08-26     7.0            7.0           0.0    2.0
      2015-08-25     7.0            7.0           0.0    1.0
      2015-08-24     7.0            7.0           0.0    0.0
      2015-08-23     7.0            7.0           0.0    0.0
      2015-08-22     7.0            7.0           0.0    0.0
      2015-08-21     7.0            7.0           0.0    1.0
      2015-08-20     7.0            7.0           0.0    2.0
      2015-08-19     7.0            7.0           0.0    3.0
...                  ...            ...           ...    ...
1115  2013-01-30  7805.0            0.0           0.0    2.0
      2013-01-29  7805.0            0.0           0.0    1.0
      2013-01-28  7805.0            0.0           0.0    0.0
      2013-01-27  7805.0            0.0           0.0    0.0
      2013-01-26  7805.0            0.0           0.0    0.0
      2013-01-25  7805.0            0.0           0.0    1.0
      2013-01-24  7805.0            0.0           0.0    2.0
      2013-01-23  7805.0            0.0           0.0    3.0
      2013-01-22  7805.0            0.0           0.0    4.0
      2013-01-21  7805.0            0.0           0.0    5.0
      2013-01-20  7805.0            0.0           0.0    5.0
      2013-01-19  7805.0            0.0           0.0    5.0
      2013-01-18  7805.0            0.0           0.0    4.0
      2013-01-17  7805.0            0.0           0.0    3.0
      2013-01-16  7805.0            0.0           0.0    2.0
      2013-01-15  7805.0            0.0           0.0    1.0
      2013-01-14  7805.0            0.0           0.0    0.0
      2013-01-13  7805.0            0.0           0.0    0.0
      2013-01-12  7805.0            0.0           0.0    0.0
      2013-01-11  7805.0            1.0           0.0    1.0
      2013-01-10  7805.0            2.0           0.0    2.0
      2013-01-09  7805.0            3.0           0.0    3.0
      2013-01-08  7805.0            4.0           0.0    4.0
      2013-01-07  7805.0            5.0           0.0    5.0
      2013-01-06  7805.0            6.0           0.0    5.0
      2013-01-05  7805.0            7.0           0.0    5.0
      2013-01-04  7805.0            7.0           0.0    4.0
      2013-01-03  7805.0            7.0           0.0    3.0
      2013-01-02  7805.0            7.0           0.0    2.0
      2013-01-01  7805.0            7.0           1.0    1.0

[1058297 rows x 4 columns]

In [68]:
bwd.drop('Store',1,inplace=True)
bwd.reset_index(inplace=True)

#回复原来的索引

In [69]:
bwd

,Store,Date,SchoolHoliday,StateHoliday,Promo
0,1,2013-01-01,1.0,1.0,0.0
1,1,2013-01-02,2.0,1.0,0.0
2,1,2013-01-03,3.0,1.0,0.0
3,1,2013-01-04,4.0,1.0,0.0
4,1,2013-01-05,5.0,1.0,0.0
5,1,2013-01-06,6.0,1.0,0.0
6,1,2013-01-07,7.0,1.0,1.0
7,1,2013-01-08,7.0,0.0,2.0
8,1,2013-01-09,7.0,0.0,3.0
9,1,2013-01-10,7.0,0.0,4.0


In [70]:
fwd.drop('Store',1,inplace=True)
fwd.reset_index(inplace=True)

In [71]:
df

,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo
Date,,,,,,,,,,
2015-09-17,1,1,False,0,13,0,105,0,0,0
2015-09-16,1,1,False,0,12,0,104,0,0,0
2015-09-15,1,1,False,0,11,0,103,0,0,0
2015-09-14,1,1,False,0,10,0,102,0,0,0
2015-09-13,1,0,False,0,9,0,101,0,9,-1
2015-09-12,1,0,False,0,8,0,100,0,8,-2
2015-09-11,1,0,False,0,7,0,99,0,7,-3
2015-09-10,1,0,False,0,6,0,98,0,6,-4
2015-09-09,1,0,False,0,5,0,97,0,5,-5


In [72]:
df.reset_index(inplace=True)

In [73]:
df.head()

,Date,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo
0,2015-09-17,1,1,False,0,13,0,105,0,0,0
1,2015-09-16,1,1,False,0,12,0,104,0,0,0
2,2015-09-15,1,1,False,0,11,0,103,0,0,0
3,2015-09-14,1,1,False,0,10,0,102,0,0,0
4,2015-09-13,1,0,False,0,9,0,101,0,9,-1


In [74]:
df = df.merge(bwd, 'left', ['Date', 'Store'], suffixes=['', '_bw'])

In [75]:
df

,Date,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bw,Promo_bw
0,2015-09-17,1,1,False,0,13,0,105,0,0,0,0.0,0.0,4.0
1,2015-09-16,1,1,False,0,12,0,104,0,0,0,0.0,0.0,3.0
2,2015-09-15,1,1,False,0,11,0,103,0,0,0,0.0,0.0,2.0
3,2015-09-14,1,1,False,0,10,0,102,0,0,0,0.0,0.0,1.0
4,2015-09-13,1,0,False,0,9,0,101,0,9,-1,0.0,0.0,0.0
5,2015-09-12,1,0,False,0,8,0,100,0,8,-2,0.0,0.0,0.0
6,2015-09-11,1,0,False,0,7,0,99,0,7,-3,0.0,0.0,0.0
7,2015-09-10,1,0,False,0,6,0,98,0,6,-4,1.0,0.0,1.0
8,2015-09-09,1,0,False,0,5,0,97,0,5,-5,2.0,0.0,2.0
9,2015-09-08,1,0,False,0,4,0,96,0,4,-6,3.0,0.0,3.0


In [76]:
df = df.merge(fwd, 'left', ['Date', 'Store'], suffixes=['', '_fw'])

In [77]:
df

,Date,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_fw,Promo_fw
0,2015-09-17,1,1,False,0,13,0,105,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
1,2015-09-16,1,1,False,0,12,0,104,0,0,0,0.0,0.0,3.0,0.0,0.0,2.0
2,2015-09-15,1,1,False,0,11,0,103,0,0,0,0.0,0.0,2.0,0.0,0.0,3.0
3,2015-09-14,1,1,False,0,10,0,102,0,0,0,0.0,0.0,1.0,0.0,0.0,4.0
4,2015-09-13,1,0,False,0,9,0,101,0,9,-1,0.0,0.0,0.0,0.0,0.0,4.0
5,2015-09-12,1,0,False,0,8,0,100,0,8,-2,0.0,0.0,0.0,0.0,0.0,4.0
6,2015-09-11,1,0,False,0,7,0,99,0,7,-3,0.0,0.0,0.0,0.0,0.0,4.0
7,2015-09-10,1,0,False,0,6,0,98,0,6,-4,1.0,0.0,1.0,0.0,0.0,3.0
8,2015-09-09,1,0,False,0,5,0,97,0,5,-5,2.0,0.0,2.0,0.0,0.0,2.0
9,2015-09-08,1,0,False,0,4,0,96,0,4,-6,3.0,0.0,3.0,0.0,0.0,1.0


In [78]:
df.drop(columns,1,inplace=True)  #删掉columns = ["Date", "Store", "Promo", "StateHoliday", "SchoolHoliday"]

In [79]:
df

,Date,Store,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_fw,Promo_fw
0,2015-09-17,1,13,0,105,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
1,2015-09-16,1,12,0,104,0,0,0,0.0,0.0,3.0,0.0,0.0,2.0
2,2015-09-15,1,11,0,103,0,0,0,0.0,0.0,2.0,0.0,0.0,3.0
3,2015-09-14,1,10,0,102,0,0,0,0.0,0.0,1.0,0.0,0.0,4.0
4,2015-09-13,1,9,0,101,0,9,-1,0.0,0.0,0.0,0.0,0.0,4.0
5,2015-09-12,1,8,0,100,0,8,-2,0.0,0.0,0.0,0.0,0.0,4.0
6,2015-09-11,1,7,0,99,0,7,-3,0.0,0.0,0.0,0.0,0.0,4.0
7,2015-09-10,1,6,0,98,0,6,-4,1.0,0.0,1.0,0.0,0.0,3.0
8,2015-09-09,1,5,0,97,0,5,-5,2.0,0.0,2.0,0.0,0.0,2.0
9,2015-09-08,1,4,0,96,0,4,-6,3.0,0.0,3.0,0.0,0.0,1.0


In [80]:
df.to_feather(r'./rossmann/'+'df')

In [81]:
df = pd.read_feather(r'./rossmann/'+'df')

In [82]:
df["Date"].head()

0   2015-09-17
1   2015-09-16
2   2015-09-15
3   2015-09-14
4   2015-09-13
Name: Date, dtype: datetime64[ns]

In [83]:
df["Date"] = pd.to_datetime(df.Date)

In [84]:
df["Date"].head()

0   2015-09-17
1   2015-09-16
2   2015-09-15
3   2015-09-14
4   2015-09-13
Name: Date, dtype: datetime64[ns]

In [85]:
df.columns

Index(['Date', 'Store', 'AfterSchoolHoliday', 'BeforeSchoolHoliday',
       'AfterStateHoliday', 'BeforeStateHoliday', 'AfterPromo', 'BeforePromo',
       'SchoolHoliday_bw', 'StateHoliday_bw', 'Promo_bw', 'SchoolHoliday_fw',
       'StateHoliday_fw', 'Promo_fw'],
      dtype='object')

In [86]:
joined = join_df(joined, df, ['Store', 'Date'])

In [87]:
joined_test = join_df(joined_test, df, ['Store', 'Date'])

In [88]:
joined = joined[joined.Sales!=0]

In [89]:
joined.reset_index(inplace=True)
joined_test.reset_index(inplace=True)  #生成从0开始的index，便于后面embedding

In [90]:
joined_test

,index,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Year,...,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_fw,Promo_fw
0,0,1,1,4,2015-09-17,1.0,1,False,0,2015,...,105,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
1,1,2,3,4,2015-09-17,1.0,1,False,0,2015,...,105,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
2,2,3,7,4,2015-09-17,1.0,1,False,0,2015,...,115,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
3,3,4,8,4,2015-09-17,1.0,1,False,0,2015,...,115,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
4,4,5,9,4,2015-09-17,1.0,1,False,0,2015,...,105,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
5,5,6,10,4,2015-09-17,1.0,1,False,0,2015,...,115,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
6,6,7,11,4,2015-09-17,1.0,1,False,0,2015,...,115,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
7,7,8,12,4,2015-09-17,1.0,1,False,0,2015,...,115,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
8,8,9,13,4,2015-09-17,1.0,1,False,0,2015,...,33,0,0,0,2.0,0.0,4.0,0.0,0.0,1.0
9,9,10,14,4,2015-09-17,1.0,1,False,0,2015,...,105,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0


## 构造特征列

In [91]:
joined.to_feather(r'./rossmann/'+'joined')
joined_test.to_feather(r'./rossmann/'+'joined_test')

In [92]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw']

contin_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']


In [93]:
len(contin_vars)

16

In [94]:
dep = 'Sales'

In [95]:
joined = joined[cat_vars+contin_vars+[dep, 'Date']].copy()

In [96]:
joined

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,...,Mean_Wind_SpeedKm_h,CloudCover,trend,trend_DE,AfterStateHoliday,BeforeStateHoliday,Promo,SchoolHoliday,Sales,Date
0,1,5,2015,7,31,False,24,0,c,a,...,11,1.0,85,83,57,0,1,1,5263,2015-07-31
1,2,5,2015,7,31,False,24,25,a,a,...,11,4.0,80,83,67,0,1,1,6064,2015-07-31
2,3,5,2015,7,31,False,24,25,a,a,...,5,2.0,86,83,57,0,1,1,8314,2015-07-31
3,4,5,2015,7,31,False,24,0,c,c,...,16,6.0,74,83,67,0,1,1,13995,2015-07-31
4,5,5,2015,7,31,False,3,0,a,a,...,11,4.0,82,83,57,0,1,1,4822,2015-07-31
5,6,5,2015,7,31,False,19,0,a,a,...,11,4.0,82,83,57,0,1,1,5651,2015-07-31
6,7,5,2015,7,31,False,24,0,a,c,...,13,6.0,77,83,67,0,1,1,15344,2015-07-31
7,8,5,2015,7,31,False,9,0,a,a,...,13,6.0,77,83,67,0,1,1,8492,2015-07-31
8,9,5,2015,7,31,False,24,0,a,c,...,5,2.0,86,83,57,0,1,1,8565,2015-07-31
9,10,5,2015,7,31,False,24,0,a,a,...,13,5.0,80,83,67,0,1,1,7185,2015-07-31


In [97]:
joined_test[dep] = 0
joined_test = joined_test[cat_vars+contin_vars+[dep, 'Date', 'Id']].copy()

In [98]:
joined_test

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,...,CloudCover,trend,trend_DE,AfterStateHoliday,BeforeStateHoliday,Promo,SchoolHoliday,Sales,Date,Id
0,1,4,2015,9,17,False,24,0,c,a,...,6.0,69,67,105,0,1,0,0,2015-09-17,1
1,3,4,2015,9,17,False,24,25,a,a,...,6.0,68,67,105,0,1,0,0,2015-09-17,2
2,7,4,2015,9,17,False,24,0,a,c,...,5.0,59,67,115,0,1,0,0,2015-09-17,3
3,8,4,2015,9,17,False,11,0,a,a,...,5.0,59,67,115,0,1,0,0,2015-09-17,4
4,9,4,2015,9,17,False,24,0,a,c,...,6.0,68,67,105,0,1,0,0,2015-09-17,5
5,10,4,2015,9,17,False,24,0,a,a,...,6.0,70,67,115,0,1,0,0,2015-09-17,6
6,11,4,2015,9,17,False,24,25,a,c,...,5.0,59,67,115,0,1,0,0,2015-09-17,7
7,12,4,2015,9,17,False,24,25,a,c,...,5.0,59,67,115,0,1,0,0,2015-09-17,8
8,13,4,2015,9,17,False,24,25,d,a,...,6.0,62,67,33,0,1,0,0,2015-09-17,9
9,14,4,2015,9,17,False,18,25,a,a,...,6.0,68,67,105,0,1,0,0,2015-09-17,10


In [99]:
for v in cat_vars: 
    joined[v] = joined[v].astype('category').cat.as_ordered()

In [100]:
joined

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,...,Mean_Wind_SpeedKm_h,CloudCover,trend,trend_DE,AfterStateHoliday,BeforeStateHoliday,Promo,SchoolHoliday,Sales,Date
0,1,5,2015,7,31,False,24,0,c,a,...,11,1.0,85,83,57,0,1,1,5263,2015-07-31
1,2,5,2015,7,31,False,24,25,a,a,...,11,4.0,80,83,67,0,1,1,6064,2015-07-31
2,3,5,2015,7,31,False,24,25,a,a,...,5,2.0,86,83,57,0,1,1,8314,2015-07-31
3,4,5,2015,7,31,False,24,0,c,c,...,16,6.0,74,83,67,0,1,1,13995,2015-07-31
4,5,5,2015,7,31,False,3,0,a,a,...,11,4.0,82,83,57,0,1,1,4822,2015-07-31
5,6,5,2015,7,31,False,19,0,a,a,...,11,4.0,82,83,57,0,1,1,5651,2015-07-31
6,7,5,2015,7,31,False,24,0,a,c,...,13,6.0,77,83,67,0,1,1,15344,2015-07-31
7,8,5,2015,7,31,False,9,0,a,a,...,13,6.0,77,83,67,0,1,1,8492,2015-07-31
8,9,5,2015,7,31,False,24,0,a,c,...,5,2.0,86,83,57,0,1,1,8565,2015-07-31
9,10,5,2015,7,31,False,24,0,a,a,...,13,5.0,80,83,67,0,1,1,7185,2015-07-31


In [101]:
def apply_cats(df, trn):
    """Changes any columns of strings in df into categorical variables using trn as
    a template for the category codes.

    Parameters:
    -----------
    df: A pandas dataframe. Any columns of strings will be changed to
        categorical values. The category codes are determined by trn.

    trn: A pandas dataframe. When creating a category for df, it looks up the
        what the category's code were in trn and makes those the category codes
        for df.

    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a

    note the type of col2 is string

    >>> train_cats(df)
    >>> df

       col1 col2
    0     1    a
    1     2    b
    2     3    a

    now the type of col2 is category {a : 1, b : 2}

    >>> df2 = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['b', 'a', 'a']})
    >>> apply_cats(df2, df)

           col1 col2
        0     1    b
        1     2    a
        2     3    a

    now the type of col is category {a : 1, b : 2}
    """
    for n,c in df.items():
        if (n in trn.columns) and (trn[n].dtype.name=='category'):
            df[n] = pd.Categorical(c, categories=trn[n].cat.categories, ordered=True)

In [102]:
apply_cats(joined_test, joined)

In [103]:
joined_test

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,...,CloudCover,trend,trend_DE,AfterStateHoliday,BeforeStateHoliday,Promo,SchoolHoliday,Sales,Date,Id
0,1,4,2015,9,17,False,24,0,c,a,...,6.0,69,67,105,0,1,0,0,2015-09-17,1
1,3,4,2015,9,17,False,24,25,a,a,...,6.0,68,67,105,0,1,0,0,2015-09-17,2
2,7,4,2015,9,17,False,24,0,a,c,...,5.0,59,67,115,0,1,0,0,2015-09-17,3
3,8,4,2015,9,17,False,11,0,a,a,...,5.0,59,67,115,0,1,0,0,2015-09-17,4
4,9,4,2015,9,17,False,24,0,a,c,...,6.0,68,67,105,0,1,0,0,2015-09-17,5
5,10,4,2015,9,17,False,24,0,a,a,...,6.0,70,67,115,0,1,0,0,2015-09-17,6
6,11,4,2015,9,17,False,24,25,a,c,...,5.0,59,67,115,0,1,0,0,2015-09-17,7
7,12,4,2015,9,17,False,24,25,a,c,...,5.0,59,67,115,0,1,0,0,2015-09-17,8
8,13,4,2015,9,17,False,24,25,d,a,...,6.0,62,67,33,0,1,0,0,2015-09-17,9
9,14,4,2015,9,17,False,18,25,a,a,...,6.0,68,67,105,0,1,0,0,2015-09-17,10


In [104]:
for v in contin_vars:
    joined[v] = joined[v].fillna(0).astype('float32')
    joined_test[v] = joined_test[v].fillna(0).astype('float32')

 ## 建立模型过程

In [105]:
#
def get_cv_idxs(n, cv_idx=0, val_pct=0.2, seed=42):
    """ Get a list of index values for Validation set from a dataset
    
    Arguments:
        n : int, Total number of elements in the data set.
        cv_idx : int, starting index [idx_start = cv_idx*int(val_pct*n)] 
        val_pct : (int, float), validation set percentage 
        seed : seed value for RandomState
        
    Returns:
        list of indexes 
    """
    np.random.seed(seed)
    n_val = int(val_pct*n)
    idx_start = cv_idx*n_val
    idxs = np.random.permutation(n)
    return idxs[idx_start:idx_start+n_val]

In [106]:
n = len(joined)
n

844338

In [107]:
idxs = get_cv_idxs(n, val_pct=150000/n)
joined_samp = joined.iloc[idxs].set_index("Date")
samp_size = len(joined_samp); samp_size

150000

In [108]:
joined_samp

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,...,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,CloudCover,trend,trend_DE,AfterStateHoliday,BeforeStateHoliday,Promo,SchoolHoliday,Sales
Date,,,,,,,,,,,,,,,,,,,,,
2015-06-16,257,2,2015,6,16,False,24,0,a,a,...,18.0,8.0,6.0,63.0,71.0,22.0,0.0,1.0,0.0,5934
2013-10-30,4,3,2013,10,30,False,24,0,c,c,...,29.0,16.0,2.0,74.0,66.0,27.0,-56.0,0.0,0.0,7800
2014-11-05,593,3,2014,11,5,False,24,0,a,c,...,14.0,8.0,7.0,75.0,71.0,33.0,-50.0,1.0,0.0,9111
2013-06-19,1026,3,2013,6,19,False,24,25,c,a,...,26.0,10.0,6.0,58.0,59.0,20.0,-106.0,1.0,0.0,7831
2014-04-28,19,1,2014,4,28,False,24,25,a,c,...,13.0,6.0,6.0,74.0,71.0,7.0,-3.0,1.0,0.0,10046
2013-10-30,52,3,2013,10,30,False,0,25,d,c,...,16.0,8.0,5.0,62.0,66.0,27.0,-2.0,0.0,1.0,6819
2015-03-20,690,5,2015,3,20,False,24,0,a,a,...,13.0,5.0,7.0,67.0,63.0,78.0,-14.0,1.0,0.0,5570
2015-04-13,469,1,2015,4,13,False,24,0,c,c,...,42.0,23.0,5.0,56.0,65.0,7.0,-18.0,1.0,0.0,15060
2014-06-28,519,6,2014,6,28,False,24,25,c,c,...,24.0,8.0,4.0,70.0,77.0,19.0,-97.0,0.0,0.0,5856


In [109]:
samp_size = n
joined_samp = joined.set_index("Date")

In [110]:
joined_samp

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,...,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,CloudCover,trend,trend_DE,AfterStateHoliday,BeforeStateHoliday,Promo,SchoolHoliday,Sales
Date,,,,,,,,,,,,,,,,,,,,,
2015-07-31,1,5,2015,7,31,False,24,0,c,a,...,24.0,11.0,1.0,85.0,83.0,57.0,0.0,1.0,1.0,5263
2015-07-31,2,5,2015,7,31,False,24,25,a,a,...,14.0,11.0,4.0,80.0,83.0,67.0,0.0,1.0,1.0,6064
2015-07-31,3,5,2015,7,31,False,24,25,a,a,...,14.0,5.0,2.0,86.0,83.0,57.0,0.0,1.0,1.0,8314
2015-07-31,4,5,2015,7,31,False,24,0,c,c,...,23.0,16.0,6.0,74.0,83.0,67.0,0.0,1.0,1.0,13995
2015-07-31,5,5,2015,7,31,False,3,0,a,a,...,14.0,11.0,4.0,82.0,83.0,57.0,0.0,1.0,1.0,4822
2015-07-31,6,5,2015,7,31,False,19,0,a,a,...,14.0,11.0,4.0,82.0,83.0,57.0,0.0,1.0,1.0,5651
2015-07-31,7,5,2015,7,31,False,24,0,a,c,...,26.0,13.0,6.0,77.0,83.0,67.0,0.0,1.0,1.0,15344
2015-07-31,8,5,2015,7,31,False,9,0,a,a,...,26.0,13.0,6.0,77.0,83.0,67.0,0.0,1.0,1.0,8492
2015-07-31,9,5,2015,7,31,False,24,0,a,c,...,14.0,5.0,2.0,86.0,83.0,57.0,0.0,1.0,1.0,8565


In [111]:
def get_sample(df,n):
    """ Gets a random sample of n rows from df, without replacement.

    Parameters:
    -----------
    df: A pandas data frame, that you wish to sample from.
    n: The number of rows you wish to sample.

    Returns:
    --------
    return value: A random sample of n rows of df.

    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a

    >>> get_sample(df, 2)
       col1 col2
    1     2    b
    2     3    a
    """
    idxs = sorted(np.random.permutation(len(df))[:n])
    return df.iloc[idxs].copy()

In [112]:
def fix_missing(df, col, name, na_dict):
    """ Fill missing data in a column of df with the median, and add a {name}_na column
    which specifies if the data was missing.

    Parameters:
    -----------
    df: The data frame that will be changed.

    col: The column of data to fix by filling in missing data.

    name: The name of the new filled column in df.

    na_dict: A dictionary of values to create na's of and the value to insert. If
        name is not a key of na_dict the median will fill any missing data. Also
        if name is not a key of na_dict and there is no missing data in col, then
        no {name}_na column is not created.


    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, np.NaN, 3], 'col2' : [5, 2, 2]})
    >>> df
       col1 col2
    0     1    5
    1   nan    2
    2     3    2

    >>> fix_missing(df, df['col1'], 'col1', {})
    >>> df
       col1 col2 col1_na
    0     1    5   False
    1     2    2    True
    2     3    2   False


    >>> df = pd.DataFrame({'col1' : [1, np.NaN, 3], 'col2' : [5, 2, 2]})
    >>> df
       col1 col2
    0     1    5
    1   nan    2
    2     3    2

    >>> fix_missing(df, df['col2'], 'col2', {})
    >>> df
       col1 col2
    0     1    5
    1   nan    2
    2     3    2


    >>> df = pd.DataFrame({'col1' : [1, np.NaN, 3], 'col2' : [5, 2, 2]})
    >>> df
       col1 col2
    0     1    5
    1   nan    2
    2     3    2

    >>> fix_missing(df, df['col1'], 'col1', {'col1' : 500})
    >>> df
       col1 col2 col1_na
    0     1    5   False
    1   500    2    True
    2     3    2   False
    """
    if is_numeric_dtype(col):
        if pd.isnull(col).sum() or (name in na_dict):
            df[name+'_na'] = pd.isnull(col)
            filler = na_dict[name] if name in na_dict else col.median()
            df[name] = col.fillna(filler)
            na_dict[name] = filler
    return na_dict


In [113]:
def scale_vars(df, mapper):
    warnings.filterwarnings('ignore', category=sklearn.exceptions.DataConversionWarning)
    if mapper is None:
        map_f = [([n],StandardScaler()) for n in df.columns if is_numeric_dtype(df[n])]
        mapper = DataFrameMapper(map_f).fit(df)
    df[mapper.transformed_names_] = mapper.transform(df)
    return mapper

In [114]:
def numericalize(df, col, name, max_n_cat):
    """ Changes the column col from a categorical type to it's integer codes.

    Parameters:
    -----------
    df: A pandas dataframe. df[name] will be filled with the integer codes from
        col.

    col: The column you wish to change into the categories.
    name: The column name you wish to insert into df. This column will hold the
        integer codes.

    max_n_cat: If col has more categories than max_n_cat it will not change the
        it to its integer codes. If max_n_cat is None, then col will always be
        converted.

    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a

    note the type of col2 is string

    >>> train_cats(df)
    >>> df

       col1 col2
    0     1    a
    1     2    b
    2     3    a

    now the type of col2 is category { a : 1, b : 2}

    >>> numericalize(df, df['col2'], 'col3', None)

       col1 col2 col3
    0     1    a    1
    1     2    b    2
    2     3    a    1
    """
    if not is_numeric_dtype(col) and ( max_n_cat is None or len(col.cat.categories)>max_n_cat):
        df[name] = col.cat.codes+1

In [115]:
def proc_df(df, y_fld=None, skip_flds=None, ignore_flds=None, do_scale=False, na_dict=None,
            preproc_fn=None, max_n_cat=None, subset=None, mapper=None):
    """ proc_df takes a data frame df and splits off the response variable, and
    changes the df into an entirely numeric dataframe.

    Parameters:
    -----------
    df: The data frame you wish to process.

    y_fld: The name of the response variable

    skip_flds: A list of fields that dropped from df.

    ignore_flds: A list of fields that are ignored during processing.

    do_scale: Standardizes each column in df. Takes Boolean Values(True,False)

    na_dict: a dictionary of na columns to add. Na columns are also added if there
        are any missing values.

    preproc_fn: A function that gets applied to df.

    max_n_cat: The maximum number of categories to break into dummy values, instead
        of integer codes.

    subset: Takes a random subset of size subset from df.

    mapper: If do_scale is set as True, the mapper variable
        calculates the values used for scaling of variables during training time (mean and standard deviation).

    Returns:
    --------
    [x, y, nas, mapper(optional)]:

        x: x is the transformed version of df. x will not have the response variable
            and is entirely numeric.

        y: y is the response variable

        nas: returns a dictionary of which nas it created, and the associated median.

        mapper: A DataFrameMapper which stores the mean and standard deviation of the corresponding continuous
        variables which is then used for scaling of during test-time.

    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a

    note the type of col2 is string

    >>> train_cats(df)
    >>> df

       col1 col2
    0     1    a
    1     2    b
    2     3    a

    now the type of col2 is category { a : 1, b : 2}

    >>> x, y, nas = proc_df(df, 'col1')
    >>> x

       col2
    0     1
    1     2
    2     1

    >>> data = DataFrame(pet=["cat", "dog", "dog", "fish", "cat", "dog", "cat", "fish"],
                 children=[4., 6, 3, 3, 2, 3, 5, 4],
                 salary=[90, 24, 44, 27, 32, 59, 36, 27])

    >>> mapper = DataFrameMapper([(:pet, LabelBinarizer()),
                          ([:children], StandardScaler())])

    >>>round(fit_transform!(mapper, copy(data)), 2)

    8x4 Array{Float64,2}:
    1.0  0.0  0.0   0.21
    0.0  1.0  0.0   1.88
    0.0  1.0  0.0  -0.63
    0.0  0.0  1.0  -0.63
    1.0  0.0  0.0  -1.46
    0.0  1.0  0.0  -0.63
    1.0  0.0  0.0   1.04
    0.0  0.0  1.0   0.21
    """
    if not ignore_flds: ignore_flds=[]
    if not skip_flds: skip_flds=[]
    if subset: df = get_sample(df,subset)
    else: df = df.copy()
    ignored_flds = df.loc[:, ignore_flds]
    df.drop(ignore_flds, axis=1, inplace=True)
    if preproc_fn: preproc_fn(df)
    if y_fld is None: y = None
    else:
        if not is_numeric_dtype(df[y_fld]): df[y_fld] = df[y_fld].cat.codes
        y = df[y_fld].values
        skip_flds += [y_fld]
    df.drop(skip_flds, axis=1, inplace=True)

    if na_dict is None: na_dict = {}
    else: na_dict = na_dict.copy()
    na_dict_initial = na_dict.copy()
    for n,c in df.items(): na_dict = fix_missing(df, c, n, na_dict)
    if len(na_dict_initial.keys()) > 0:
        df.drop([a + '_na' for a in list(set(na_dict.keys()) - set(na_dict_initial.keys()))], axis=1, inplace=True)
    if do_scale: mapper = scale_vars(df, mapper)
    for n,c in df.items(): numericalize(df, c, n, max_n_cat)
    df = pd.get_dummies(df, dummy_na=True)
    df = pd.concat([ignored_flds, df], axis=1)
    res = [df, y, na_dict]
    if do_scale: res = res + [mapper]
    return res

In [123]:
df, y, nas, mapper = proc_df(joined_samp, 'Sales', do_scale=True)
yl = np.log(y)

3.828641396489095

In [117]:
from pandas.api.types import is_string_dtype, is_numeric_dtype

In [118]:
import warnings

In [119]:
import sklearn

In [120]:
from sklearn.preprocessing import LabelEncoder, Imputer, StandardScaler

In [121]:
import sklearn_pandas

In [122]:
from sklearn_pandas import DataFrameMapper

In [126]:
df

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,...,Min_Humidity,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,CloudCover,trend,trend_DE,AfterStateHoliday,BeforeStateHoliday,Promo,SchoolHoliday
Date,,,,,,,,,,,,,,,,,,,,,
2015-07-31,1,5,3,7,31,1,25,1,3,1,...,-1.620066,0.149027,-0.142774,-1.844823,1.732492,1.724334,0.604461,1.131120,1.113717,2.041050
2015-07-31,2,5,3,7,31,1,25,26,1,1,...,-1.264031,-0.960613,-0.142774,-0.488722,1.294578,1.724334,0.926957,1.131120,1.113717,2.041050
2015-07-31,3,5,3,7,31,1,25,26,1,1,...,-1.314893,-0.960613,-1.154031,-1.392789,1.820074,1.724334,0.604461,1.131120,1.113717,2.041050
2015-07-31,4,5,3,7,31,1,25,1,3,3,...,-1.009721,0.038063,0.699941,0.415345,0.769081,1.724334,0.926957,1.131120,1.113717,2.041050
2015-07-31,5,5,3,7,31,1,4,1,1,1,...,-1.213169,-0.960613,-0.142774,-0.488722,1.469743,1.724334,0.604461,1.131120,1.113717,2.041050
2015-07-31,6,5,3,7,31,1,20,1,1,1,...,-1.213169,-0.960613,-0.142774,-0.488722,1.469743,1.724334,0.604461,1.131120,1.113717,2.041050
2015-07-31,7,5,3,7,31,1,25,1,1,3,...,-0.857134,0.370955,0.194312,0.415345,1.031829,1.724334,0.926957,1.131120,1.113717,2.041050
2015-07-31,8,5,3,7,31,1,10,1,1,1,...,-0.857134,0.370955,0.194312,0.415345,1.031829,1.724334,0.926957,1.131120,1.113717,2.041050
2015-07-31,9,5,3,7,31,1,25,1,1,3,...,-1.314893,-0.960613,-1.154031,-1.392789,1.820074,1.724334,0.604461,1.131120,1.113717,2.041050


In [127]:
df[]

array([5263, 6064, 8314, ..., 5035, 4491, 5961], dtype=int64)

In [128]:
nas

{}

In [129]:
mapper

DataFrameMapper(default=False, df_out=False,
        features=[(['CompetitionDistance'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['Max_TemperatureC'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['Mean_TemperatureC'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['Min_TemperatureC'], StandardScaler(co...rue, with_std=True)), (['SchoolHoliday'], StandardScaler(copy=True, with_mean=True, with_std=True))],
        input_df=False, sparse=False)

In [130]:
joined_test = joined_test.set_index("Date")

In [131]:
df_test, _, nas, mapper = proc_df(joined_test, 'Sales', do_scale=True, skip_flds=['Id'],
                                  mapper=mapper, na_dict=nas)

In [132]:
train_ratio = 0.75
# train_ratio = 0.9
train_size = int(samp_size * train_ratio); train_size
val_idx = list(range(train_size, len(df)))


#训练集的后面0.75当作验证集

In [133]:
train_size

633253

In [134]:
len(val_idx)

211085

In [135]:
def inv_y(a): 
    return np.exp(a)
#返回e的a次方

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

In [136]:
max_log_y

10.634676867382668

In [137]:
cat_sz = [(c, len(joined_samp[c].cat.categories)+1) for c in cat_vars]

In [138]:
cat_sz

[('Store', 1116),
 ('DayOfWeek', 8),
 ('Year', 4),
 ('Month', 13),
 ('Day', 32),
 ('StateHoliday', 3),
 ('CompetitionMonthsOpen', 26),
 ('Promo2Weeks', 27),
 ('StoreType', 5),
 ('Assortment', 4),
 ('PromoInterval', 4),
 ('CompetitionOpenSinceYear', 23),
 ('Promo2SinceYear', 9),
 ('State', 13),
 ('Week', 53),
 ('Events', 22),
 ('Promo_fw', 7),
 ('Promo_bw', 7),
 ('StateHoliday_fw', 4),
 ('StateHoliday_bw', 4),
 ('SchoolHoliday_fw', 9),
 ('SchoolHoliday_bw', 9)]

In [139]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]   #计算每个embedding vector的长度

In [140]:
emb_szs

[(1116, 50),
 (8, 4),
 (4, 2),
 (13, 7),
 (32, 16),
 (3, 2),
 (26, 13),
 (27, 14),
 (5, 3),
 (4, 2),
 (4, 2),
 (23, 12),
 (9, 5),
 (13, 7),
 (53, 27),
 (22, 11),
 (7, 4),
 (7, 4),
 (4, 2),
 (4, 2),
 (9, 5),
 (9, 5)]

In [142]:
len(emb_szs)

22

In [148]:
df[cat_vars]

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,...,Promo2SinceYear,State,Week,Events,Promo_fw,Promo_bw,StateHoliday_fw,StateHoliday_bw,SchoolHoliday_fw,SchoolHoliday_bw
Date,,,,,,,,,,,,,,,,,,,,,
2015-07-31,1,5,3,7,31,1,25,1,3,1,...,1,5,31,1,6,6,1,1,8,6
2015-07-31,2,5,3,7,31,1,25,26,1,1,...,3,12,31,1,2,6,1,1,2,6
2015-07-31,3,5,3,7,31,1,25,26,1,1,...,4,7,31,1,6,6,1,1,6,6
2015-07-31,4,5,3,7,31,1,25,1,3,3,...,1,1,31,0,2,6,1,1,2,6
2015-07-31,5,5,3,7,31,1,4,1,1,1,...,1,10,31,0,2,6,1,1,2,6
2015-07-31,6,5,3,7,31,1,20,1,1,1,...,1,10,31,0,2,6,1,1,2,6
2015-07-31,7,5,3,7,31,1,25,1,1,3,...,1,9,31,11,6,6,1,1,6,6
2015-07-31,8,5,3,7,31,1,10,1,1,1,...,1,9,31,11,6,6,1,1,6,6
2015-07-31,9,5,3,7,31,1,25,1,1,3,...,1,7,31,1,6,6,1,1,6,6


In [151]:
#embedding层拼接
init = keras.initializers.RandomUniform(minval=-1, maxval=1, seed=None)

e1 = Input(shape=[22]) # None*1
# e2 = Input(shape=[1]) # None*1
# e3 = Input(shape=[1])
# e4 = Input(shape=[1])
# e5 = Input(shape=[1])
# e6 = Input(shape=[1])
# e7 = Input(shape=[1])
# e8 = Input(shape=[1])
# e9 = Input(shape=[1])
# e10 = Input(shape=[1])
# e11 = Input(shape=[1])
# e12 = Input(shape=[1])
# e13 = Input(shape=[1])
# e14 = Input(shape=[1])
# e15 = Input(shape=[1])
# e16 = Input(shape=[1])
# e17 = Input(shape=[1])
# e18 = Input(shape=[1])
# e19 = Input(shape=[1])
# e20 = Input(shape=[1])
# e21 = Input(shape=[1])
# e22 = Input(shape=[1])

emb1 = Embedding(1116, 50, embeddings_initializer=init, mask_zero=False,trainable=False)(e1) # None*3*5
# emb2 = Embedding(8, 4, embeddings_initializer=init, mask_zero=False,trainable=False)(e2) # None*3*5
# emb3 = Embedding(4, 2, embeddings_initializer=init, mask_zero=False,trainable=False)(e3)
# emb4 = Embedding(13, 7, embeddings_initializer=init, mask_zero=False,trainable=False)(e4)
# emb5 = Embedding(32, 16, embeddings_initializer=init, mask_zero=False,trainable=False)(e5)
# emb6 = Embedding(3, 2, embeddings_initializer=init, mask_zero=False,trainable=False)(e6)
# emb7 = Embedding(26, 13, embeddings_initializer=init, mask_zero=False,trainable=False)(e7)
# emb8 = Embedding(27, 14, embeddings_initializer=init, mask_zero=False,trainable=False)(e8)
# emb9 = Embedding(5, 3, embeddings_initializer=init, mask_zero=False,trainable=False)(e9)
# emb10 = Embedding(4, 2, embeddings_initializer=init, mask_zero=False,trainable=False)(e10)
# emb11 = Embedding(4, 2, embeddings_initializer=init, mask_zero=False,trainable=False)(e11)
# emb12 = Embedding(24, 12, embeddings_initializer=init, mask_zero=False,trainable=False)(e12)
# emb13 = Embedding(9, 5, embeddings_initializer=init, mask_zero=False,trainable=False)(e13)
# emb14 = Embedding(13, 7, embeddings_initializer=init, mask_zero=False,trainable=False)(e14)
# emb15 = Embedding(53, 27, embeddings_initializer=init, mask_zero=False,trainable=False)(e15)
# emb16 = Embedding(22, 11, embeddings_initializer=init, mask_zero=False,trainable=False)(e16)
# emb17 = Embedding(7, 4, embeddings_initializer=init, mask_zero=False,trainable=False)(e17)
# emb18 = Embedding(7, 4, embeddings_initializer=init, mask_zero=False,trainable=False)(e18)
# emb19 = Embedding(4, 2, embeddings_initializer=init, mask_zero=False,trainable=False)(e19)
# emb20 = Embedding(4, 2, embeddings_initializer=init, mask_zero=False,trainable=False)(e20)
# emb21 = Embedding(9, 5, embeddings_initializer=init, mask_zero=False,trainable=False)(e21)
# emb22 = Embedding(9, 5, embeddings_initializer=init, mask_zero=False,trainable=False)(e22)

auxiliary_input = Input(shape=(1,16), name='con')

# x  = Concatenate(axis=-1)([emb1,emb2,emb3,emb4,emb5,emb6,emb7,emb8,emb9,emb10
#                            ,emb11,emb12,emb13,emb14,emb15,emb16,emb17,emb18,emb19,emb20,emb21,emb22,auxiliary_input])

In [152]:
lstm_out = LSTM(32)(emb1)

In [145]:
#lstm_out = LSTM(32)(main_input)

In [153]:
auxiliary_input = Input(shape=(16,), name='aux_input')


In [146]:
#lstm_out

In [155]:
x = keras.layers.concatenate([lstm_out, auxiliary_input])


In [156]:
x = Dense(64, activation='tanh')(x)
x = Dense(64, activation='tanh')(x)
x = Dense(64, activation='tanh')(x)

In [147]:
#auxiliary_input = Input(shape=(16,), name='con')

In [148]:
#auxiliary_input

In [149]:
#x = keras.layers.concatenate([lstm_out, auxiliary_input])

In [157]:
x

<tf.Tensor 'dense_4/Tanh:0' shape=(?, 64) dtype=float32>

In [158]:
main_output = Dense(1,activation = 'linear',name='main_output')(x)

In [154]:
#model = Model(inputs=[e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,e11,e12,e13,e14,e15,e16,e17,e18,e19,e20,e21,e22,auxiliary_input], outputs = main_output )

In [159]:
model = Model(inputs=[e1, auxiliary_input], outputs=main_output)


In [160]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 22)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 22, 50)       55800       input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 32)           10624       embedding_2[0][0]                
__________________________________________________________________________________________________
aux_input (InputLayer)          (None, 16)           0                                            
__________________________________________________________________________________________________
concatenat

In [161]:
main_output

<tf.Tensor 'main_output/BiasAdd:0' shape=(?, 1) dtype=float32>

In [162]:
model.compile(optimizer='Adam', loss='mean_squared_error')

In [163]:
model

In [159]:
df

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,...,Min_Humidity,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,CloudCover,trend,trend_DE,AfterStateHoliday,BeforeStateHoliday,Promo,SchoolHoliday
Date,,,,,,,,,,,,,,,,,,,,,
2015-07-31,1,5,3,7,31,1,25,1,3,1,...,-1.620066,0.149027,-0.142774,-1.844823,1.732492,1.724334,0.604461,1.131120,1.113717,2.041050
2015-07-31,2,5,3,7,31,1,25,26,1,1,...,-1.264031,-0.960613,-0.142774,-0.488722,1.294578,1.724334,0.926957,1.131120,1.113717,2.041050
2015-07-31,3,5,3,7,31,1,25,26,1,1,...,-1.314893,-0.960613,-1.154031,-1.392789,1.820074,1.724334,0.604461,1.131120,1.113717,2.041050
2015-07-31,4,5,3,7,31,1,25,1,3,3,...,-1.009721,0.038063,0.699941,0.415345,0.769081,1.724334,0.926957,1.131120,1.113717,2.041050
2015-07-31,5,5,3,7,31,1,4,1,1,1,...,-1.213169,-0.960613,-0.142774,-0.488722,1.469743,1.724334,0.604461,1.131120,1.113717,2.041050
2015-07-31,6,5,3,7,31,1,20,1,1,1,...,-1.213169,-0.960613,-0.142774,-0.488722,1.469743,1.724334,0.604461,1.131120,1.113717,2.041050
2015-07-31,7,5,3,7,31,1,25,1,1,3,...,-0.857134,0.370955,0.194312,0.415345,1.031829,1.724334,0.926957,1.131120,1.113717,2.041050
2015-07-31,8,5,3,7,31,1,10,1,1,1,...,-0.857134,0.370955,0.194312,0.415345,1.031829,1.724334,0.926957,1.131120,1.113717,2.041050
2015-07-31,9,5,3,7,31,1,25,1,1,3,...,-1.314893,-0.960613,-1.154031,-1.392789,1.820074,1.724334,0.604461,1.131120,1.113717,2.041050


In [167]:
model.fit([train_cat,train_con], yl,validation_split=0.3,epochs=3, batch_size=32)

Train on 591036 samples, validate on 253302 samples
Epoch 1/3
591036/591036 [==============================] - 821s 1ms/step - loss: 0.1296 - val_loss: 0.0485
Epoch 2/3
591036/591036 [==============================] - 804s 1ms/step - loss: 0.0263 - val_loss: 0.0460
Epoch 3/3
591036/591036 [==============================] - 809s 1ms/step - loss: 0.0207 - val_loss: 0.0514


In [164]:
train_cat = df[cat_vars]

In [189]:
train_cat

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,...,Promo2SinceYear,State,Week,Events,Promo_fw,Promo_bw,StateHoliday_fw,StateHoliday_bw,SchoolHoliday_fw,SchoolHoliday_bw
Date,,,,,,,,,,,,,,,,,,,,,
2015-07-31,1,5,3,7,31,1,25,1,3,1,...,1,5,31,1,6,6,1,1,8,6
2015-07-31,2,5,3,7,31,1,25,26,1,1,...,3,12,31,1,2,6,1,1,2,6
2015-07-31,3,5,3,7,31,1,25,26,1,1,...,4,7,31,1,6,6,1,1,6,6
2015-07-31,4,5,3,7,31,1,25,1,3,3,...,1,1,31,0,2,6,1,1,2,6
2015-07-31,5,5,3,7,31,1,4,1,1,1,...,1,10,31,0,2,6,1,1,2,6
2015-07-31,6,5,3,7,31,1,20,1,1,1,...,1,10,31,0,2,6,1,1,2,6
2015-07-31,7,5,3,7,31,1,25,1,1,3,...,1,9,31,11,6,6,1,1,6,6
2015-07-31,8,5,3,7,31,1,10,1,1,1,...,1,9,31,11,6,6,1,1,6,6
2015-07-31,9,5,3,7,31,1,25,1,1,3,...,1,7,31,1,6,6,1,1,6,6


In [165]:
train_con = df[contin_vars]

In [166]:
train_con

,CompetitionDistance,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,CloudCover,trend,trend_DE,AfterStateHoliday,BeforeStateHoliday,Promo,SchoolHoliday
Date,,,,,,,,,,,,,,,,
2015-07-31,-0.534831,1.008113,0.806978,0.317856,0.614244,-1.507606,-1.620066,0.149027,-0.142774,-1.844823,1.732492,1.724334,0.604461,1.131120,1.113717,2.041050
2015-07-31,-0.624524,0.538548,0.394117,0.163376,0.873262,-0.902093,-1.264031,-0.960613,-0.142774,-0.488722,1.294578,1.724334,0.926957,1.131120,1.113717,2.041050
2015-07-31,1.112961,0.773330,0.394117,0.008896,0.873262,-0.977782,-1.314893,-0.960613,-1.154031,-1.392789,1.820074,1.724334,0.604461,1.131120,1.113717,2.041050
2015-07-31,-0.618118,0.538548,0.531737,0.472336,0.096209,-0.977782,-1.009721,0.038063,0.699941,0.415345,0.769081,1.724334,0.926957,1.131120,1.113717,2.041050
2015-07-31,3.134902,0.655939,0.669357,0.626816,-1.457898,-1.431917,-1.213169,-0.960613,-0.142774,-0.488722,1.469743,1.724334,0.604461,1.131120,1.113717,2.041050
2015-07-31,-0.657839,0.655939,0.669357,0.626816,-1.457898,-1.431917,-1.213169,-0.960613,-0.142774,-0.488722,1.469743,1.724334,0.604461,1.131120,1.113717,2.041050
2015-07-31,2.377635,0.421157,0.531737,0.317856,-0.680845,-0.977782,-0.857134,0.370955,0.194312,0.415345,1.031829,1.724334,0.926957,1.131120,1.113717,2.041050
2015-07-31,0.266001,0.421157,0.531737,0.317856,-0.680845,-0.977782,-0.857134,0.370955,0.194312,0.415345,1.031829,1.724334,0.926957,1.131120,1.113717,2.041050
2015-07-31,-0.437450,0.773330,0.394117,0.008896,0.873262,-0.977782,-1.314893,-0.960613,-1.154031,-1.392789,1.820074,1.724334,0.604461,1.131120,1.113717,2.041050
